In [ ]:
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import datetime as dt
import random
from skimage import transform
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import TimeDistributed, Dense, MaxPooling2D, Dropout, Conv2D, LSTM, Flatten, RandomFlip, RandomRotation, Resizing,Rescaling
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
%matplotlib inline

In [ ]:

from tensorflow.keras.models import load_model
aifit_lrcn_model = load_model('aifit_lrcn.h5')
aifit_lrcn_model.summary()

In [ ]:
import mediapipe as mp
# Initialize mediapipe pose class.
mp_pose = mp.solutions.pose

# Setup the Pose function for images - independently for the images standalone processing.
pose_image = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

# Setup the Pose function for videos - for video processing.
pose_video = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.7,
                          min_tracking_confidence=0.7)

# Initialize mediapipe drawing class - to draw the landmarks points.
mp_drawing = mp.solutions.drawing_utils
def detectPose(image_pose, pose, draw=False, display=False):
    image_in_RGB = cv2.cvtColor(image_pose, cv2.COLOR_BGR2RGB)
    resultant = pose.process(image_in_RGB)
    marked_img = cv2.cvtColor(image_pose, cv2.COLOR_BGR2GRAY)
    marked_img = cv2.applyColorMap(marked_img, cv2.COLORMAP_HOT)
    if resultant.pose_landmarks and draw:    
        mp_drawing.draw_landmarks(image=marked_img, landmark_list=resultant.pose_landmarks,
                                  connections=mp_pose.POSE_CONNECTIONS,
                                  landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255,255,255),
                                                                               thickness=1, circle_radius=1),
                                  connection_drawing_spec=mp_drawing.DrawingSpec(color=(255,255,0),
                                                                               thickness=1, circle_radius=1))

    if display:
            plt.figure(figsize=[5,5])
            plt.subplot(121);plt.imshow(image_pose[:,:,::-1]);plt.title("Input Image");plt.axis('off');
            plt.subplot(122);plt.imshow(marked_img[:,:,::-1]);plt.title("Pose detected Image");plt.axis('off');

    else:
        return image_pose, marked_img

In [ ]:
from collections import deque 
SEQUENCE_LENGTH = 20
IMAGE_HEIGHT, IMAGE_WIDTH = 64, 64
CLASSES_LIST = ['Armraise', 'Bicep Curl', 'Bird Dog', 'Fly', 'Overhead Press', 'Pushup', 'Squat']
video_reader = cv2.VideoCapture(0)
original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))
frames_queue = deque(maxlen = SEQUENCE_LENGTH)

predicted_class_name = ''
confidence = 0
while video_reader.isOpened():
    ok, frame = video_reader.read() 
    
    if not ok:
        break
        
    # Resize the Frame to fixed Dimensions.
    image_frame, marked_frame = detectPose(frame, pose_image, draw=True, display=False)
    resized_frame = cv2.resize(marked_frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
    print(np.shape(frames_queue))
    frames_queue.append(resized_frame)
    if len(frames_queue) == SEQUENCE_LENGTH:
        predicted_labels_probabilities = aifit_lrcn_model.predict(np.expand_dims(frames_queue, axis = 0))[0]
        predicted_label = np.argmax(predicted_labels_probabilities)
        predicted_class_name = CLASSES_LIST[predicted_label]
        confidence = round(predicted_labels_probabilities[predicted_label]*100, 2)

    cv2.putText(marked_frame, f'{predicted_class_name} - {confidence}%'
                , (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('AiFit', marked_frame)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break


video_reader.release()
cv2.destroyAllWindows()